In [1]:
import pandas as pd
import requests
from time import sleep
import os

In [3]:
# token and counter id for log request
token = ''
Authorization = 'OAuth ' + token
counterId = ''

# params of log request
params = {
    ...
}

In [ ]:
# step 1: evaluating the possibility of creating a request
# https://yandex.ru/dev/metrika/doc/api2/logs/queries/evaluate.html
url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequests/evaluate'
r = requests.get(url, params=params, headers={'Authorization': Authorization})
r.json()

In [ ]:
# step 2: create log request
# https://yandex.ru/dev/metrika/doc/api2/logs/queries/createlogrequest.html

url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequests'
r = requests.post(url, params=params, headers={'Authorization': Authorization})
if r.status_code == 200:
    print(f'Код ответа: {r.status_code}')
    print('Содержимое ответа:')
    print(r.json())
    request_id = str(r.json().get('log_request').get('request_id'))
else:
    print(f'Код ответа: {r.status_code}')
    print(r.json())

In [43]:
# step 3: get info about log request
# https://yandex.ru/dev/metrika/doc/api2/logs/queries/getlogrequest.html 
try:
    while True:
        url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{request_id}'
        r = requests.get(url, headers={'Authorization': Authorization})
        if r.json().get('log_request').get('status') == 'processed':
            print('status:', r.json().get('log_request').get('status'))
            print(r.json())
            break
        sleep(20)
except Exception:
    print(Exception)

status: processed


In [ ]:
# step 4: download data
# https://yandex.ru/dev/metrika/doc/api2/logs/queries/download.html

try:
    dfs = []
    for part in r.json().get('log_request').get('parts'):
        part_number = part.get('part_number')
        url = f'https://api-metrika.yandex.net/management/v1/counter/{counterId}/logrequest/{request_id}/part/{part_number}/download'
        r = requests.get(url, headers={'Authorization': Authorization})
        load_data = r.text
        df = pd.DataFrame([x.split('\t') for x in load_data.split('\n')])
        df.columns = list(df.iloc[0])
        df = df.drop(0)
        dfs.append(df)
    data = pd.concat(dfs)
except Exception as e:
    print(Exception)

display(data.info())
data.head()